## Detección de Pishing - Laboratorio 01

Sara Zavala  18893   
Alexa Bravo  18831  

In [1]:
#Se importan las librerias que vamos a utilizar.
import  numpy as np
import  re

import pandas as pd
import  matplotlib.pyplot as plt
import seaborn as sns
from urllib.parse import urlparse
import pandas_profiling as pp
from pandas_profiling import ProfileReport

import sklearn
from sklearn import metrics, model_selection, tree

### Parte 1 – Ingeniería de características

#### Exploración de datos 

In [2]:
#Cargamos los datos.
df = pd.read_csv('dataset_pishing.csv')
df.head(5)

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,http://www.crestonwood.com/router.php,0,1,0,0,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,1,0,0,0,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,phishing
2,https://support-appleld.com.secureupdate.duila...,1,0,1,0,0,0,0,0,1,...,1,0,0,14,4004,5828815,0,1,0,phishing
3,http://rgipt.ac.in,0,0,0,0,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,0,1,0,0,0,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,legitimate


In [3]:
#Cantidad de datos en la columna status. 
df.status.value_counts()

legitimate    5715
phishing      5715
Name: status, dtype: int64

Se puede observar que el dataset esta balanceado ya que tiene la columna "status" cuenta con la misma cantidad de datos en la variable "legitimate" como en la variable "phishing".  

#### Derivación de características 

In [4]:
#f1 longitud del url. 
df['f1'] = df['url'].str.len()

In [5]:
#Función para obtener la longitud del hostname. 
def Hostname(s):
    return urlparse(s).hostname

#f2: Longitud del hostname.
df['f2'] = df['url'].apply(Hostname).str.len()

In [6]:
#f4-20: Caracteres especiales. 

In [7]:
#Función para verificar que el url tenga "https"
def Https(s):
    return 1 if urlparse(s).scheme == 'https' else 0

#f25: HTTPS 
df['f25'] = df['url'].apply(Https)

In [8]:
#f26: Proporción de digitos en la URL.
df['f26'] = df['url'].str.count('[0-9]')/df['f1']

In [9]:
#f27: Proporción de digitos en el hostname.
df['f27'] = df['url'].apply(Hostname).str.count('[0-9]')/df['f2']

In [10]:
#Dataset con las columnas nuevas. 
df.head(5)

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,web_traffic,dns_record,google_index,page_rank,status,f1,f2,f25,f26,f27
0,http://www.crestonwood.com/router.php,0,1,0,0,0,0,0,0,0,...,0,1,1,4,legitimate,37,19,0,0.000000,0.0
1,http://shadetreetechnology.com/V4/validation/a...,1,0,0,0,0,0,0,0,0,...,0,0,1,2,phishing,77,23,0,0.220779,0.0
2,https://support-appleld.com.secureupdate.duila...,1,0,1,0,0,0,0,0,1,...,5828815,0,1,0,phishing,126,50,1,0.150794,0.0
3,http://rgipt.ac.in,0,0,0,0,0,0,0,0,0,...,107721,0,0,3,legitimate,18,11,0,0.000000,0.0
4,http://www.iracing.com/tracks/gateway-motorspo...,0,1,0,0,0,0,0,0,0,...,8725,0,0,6,legitimate,55,15,0,0.000000,0.0


#### Preprocesamiento 

In [11]:
#Volvemos binaria la variable status. 
df['status'] = df['status'].replace(to_replace='phishing', value=1)
df['status'] = df['status'].replace(to_replace='legitimate', value=0)

In [12]:
#Eliminamos la columna "url"
df.drop(['url'], axis=1, inplace=True)
df.head(5)

,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,abnormal_subdomain,...,web_traffic,dns_record,google_index,page_rank,status,f1,f2,f25,f26,f27
0,0,1,0,0,0,0,0,0,0,0,...,0,1,1,4,0,37,19,0,0.000000,0.0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,1,2,1,77,23,0,0.220779,0.0
2,1,0,1,0,0,0,0,0,1,0,...,5828815,0,1,0,1,126,50,1,0.150794,0.0
3,0,0,0,0,0,0,0,0,0,0,...,107721,0,0,3,0,18,11,0,0.000000,0.0
4,0,1,0,0,0,0,0,0,0,0,...,8725,0,0,6,0,55,15,0,0.000000,0.0


#### Visualización de resultados 

#### Selección de Características 

### Parte 2 – Implementación del modelo 

#### Separación de datos

#### Implementación 